In [34]:
"""===============================================================================================
Overall Asymmetry Index (oAI) computation

Reference: 
1. Shu-Yen Wan, Pei-Ying Tsai, and Lun-Jou Lo, "Quantifying Perceived Facial Asymmetry to Enhance 
   Physician-Patient Communications," Applied Sciences, vol. 11, no. 18, Pages 8398, 2021. 
   DOI: 10.3390/app11188398
2. R.O.C. (Taiwan) Patent (I595430)

Copyright 2022, Shu-Yen Wan, sywan@mail.cgu.edu.tw

20 facial landmarks are considered to compute the oAI

Estimated facial landmarks - may not be identical with the locations defined by Farkas, L.G. 
(Anthropometry of the Head and Face; Raven Press: New York, NY, USA, 1994.)

01 Glabella (G) => 9
    Most prominent midline point between eyebrows
02 Nasion (N) => 168
    Deepest point of nasal bridge
03 Pronasale (Prn) => 4
    Most protruded point of the apex nasi
04 Subnasale (Sn) => 2
    Midpoint of angle at columella base
05 Labial Superius (Ls) => 0
    Midpoint of the upper vermilion line
06 Labial Inferius (Li) => 17
    Midpoint of the lower vermilion line
07 Stomion (Sto) => 13/14
    Midpoint of the mouth orifice
08 Menton (Me) => 152
    Most inferior point on chin
09 Exocanthion (Ex) => 33(r)/263(l)
    Outer commissure of the eye fissure 
10 Endocanthion (En) => 133(r)/362(l)
    Inner commissure of the eye fissure
11 Alar curvature (Al) => 129(r)/358(l)
    Most lateral point on alar contour
12 Cheilion (Ch) => 61(r)/291(l)
    Point located at lateral labial commissure
13 Zygion (Zy) => 116(r)/345(l)
    The most lateral extents of the zygomatic arches
14 Gonion (Go) => 138(r)/367(l)
    The inferior aspect of the mandibe at its most acute point
==============================================================================================="""

medial = {"Glabella":9, "Nasion":168, "Pronasale":4, "Subnasale":2, 
          "Labial Superius":0, "Labial Inferius":17, "Stomion":13, "Menton":152}
bilateral = {"Exocanthion":[33,263], "Endocanthion":[133,362], "Alar curvature":[129,358],
            "Cheilion":[61,291], "Zygion":[116,345], "Gonion":[138,367]}

weight_medial = [5.09, 5.75, 8.21, 11.11, 4.10, 7.75, 6.38, 8.62]
weight_bilateral = [5.13, 5.36, 11.71, 8.76, 5.27, 6.76]

weights = weight_medial + weight_bilateral

In [2]:
# !pip install mediapipe opencv-python
import cv2
import mediapipe as mp
import pandas as pd
import numpy as np
import os
import math

In [3]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_face_mesh = mp.solutions.face_mesh
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=5)

In [13]:
# Scaling factor
scalingFactor = 1

In [52]:
image_path = os.path.join('img')
IMAGE_FILES = [f for f in os.listdir(image_path) if os.path.isfile(os.path.join(image_path, f))]

In [6]:
# Function: bounding_box()
# Description: Determine the bounding box of given landmarks
#
def bounding_box(landmarks):   # type(landkmakrs): google.protobuf.pyext._message.RepeatedCompositeContainer
    if landmarks is None:
        print("Null landmarks")
        exit()
    upper_left = [landmarks[0].x, landmarks[0].y]
    bottom_right = [landmarks[0].x, landmarks[0].y]
    for point in landmarks:
        upper_left[0] = point.x if point.x < upper_left[0] else upper_left[0]
        upper_left[1] = point.y if point.y < upper_left[1] else upper_left[1]
        bottom_right[0] = point.x if point.x > bottom_right[0] else bottom_right[0]
        bottom_right[1] = point.y if point.y > bottom_right[1] else bottom_right[1]
    return upper_left, bottom_right

In [32]:
# Function: create_dataframe
# Description: Create dataframe with empty entries
# Return: empty dataframe
#         header list for feature coordinates
#         header list for AI and oAI
#
def create_dataframe():
    #features = ['Image'] 
    features = []
    features2 = []
    ai = []
    for c in medial.keys():
        features.append(c + '.x')
        features.append(c + '.y')
        features2.append(c)
    for c in bilateral.keys():
        features.append(c + '_right.x')
        features.append(c + '_right.y')
        features.append(c + '_left.x')
        features.append(c + '_left.y')
        features2.append(c)
    features2.append('oAI')
    return pd.DataFrame(columns=features), pd.DataFrame(columns=features2), features, features2

In [61]:
# Create an empty dataframe with default header
df, df_ai, columns, columns_AIs = create_dataframe()

with mp_face_mesh.FaceMesh(
    static_image_mode=True,
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5) as face_mesh:
    
    for idx, file in enumerate(IMAGE_FILES):
    
        # Load the image and scale to desired size
        # All images are stored in os.path.join(image_path, file)
        annotated_image = cv2.imread(os.path.join(image_path,file))
        width = int(annotated_image.shape[1]*scalingFactor)
        height = int(annotated_image.shape[0]*scalingFactor)
        annotated_image = cv2.resize(annotated_image, (width, height), interpolation=cv2.INTER_CUBIC)

        # Convert the BGR image to RGB before processing.
        results = face_mesh.process(cv2.cvtColor(annotated_image, cv2.COLOR_BGR2RGB))
        
        # Compile the dataframe data for each row
        # coord: feature coordinates
        # AIs: AI and oAI
        coord = []
        AIs = []
        
        # Print and draw face mesh landmarks on the image.
        if not results.multi_face_landmarks:
            continue
        for face_landmarks in results.multi_face_landmarks:
      
            # Determine the bounding box and de-normalize
            upper_left, bottom_right = bounding_box(face_landmarks.landmark)
            upper_left[0] = int(upper_left[0]*width)
            upper_left[1] = int(upper_left[1]*height)
            bottom_right[0] = int(bottom_right[0]*width)
            bottom_right[1] = int(bottom_right[1]*height)
            cv2.rectangle(annotated_image, upper_left, bottom_right, color=(0,0,255), thickness=2)
            
            # Exocanthion(right,left) and Nasion are explicitly extracted to perform pre-computation
            Exocanthion_right = face_landmarks.landmark[bilateral['Exocanthion'][0]]
            Exocanthion_left = face_landmarks.landmark[bilateral['Exocanthion'][1]]
            Nasion = face_landmarks.landmark[medial['Nasion']]
            Ex_r_x = int(face_landmarks.landmark[bilateral['Exocanthion'][0]].x*width)
            Ex_r_y = int(face_landmarks.landmark[bilateral['Exocanthion'][0]].y*height)
            Ex_l_x = int(face_landmarks.landmark[bilateral['Exocanthion'][1]].x*width)
            Ex_l_y = int(face_landmarks.landmark[bilateral['Exocanthion'][1]].y*height)
            N_x = int(face_landmarks.landmark[medial['Nasion']].x*width)
            N_y = int(face_landmarks.landmark[medial['Nasion']].y*height)
            
            """
            medial = {"Glabella":9, "Nasion":168, "Pronasale":4, "Subnasale":2, 
                      "Labial Superius":0, "Labial Inferius":17, "Stomion":13, "Menton":152}
            bilateral = {"Exocanthion":[33,263], "Endocanthion":[133,362], "Alar curvature":[129,358],
                        "Cheilion":[61,291], "Zygion":[116,345], "Gonion":[138,367]}
            """
          
            # slope (Sagittal normal) = (Ex_left.y-Ex_right.y)/(Ex_left.x-Ex_right.x)
            # constant_term = (N.x+(slope)*N.y)
            slope = float(Ex_l_y-Ex_r_y)/float(Ex_l_x-Ex_r_x)
            sqrt_slope_square_plus_1 = math.sqrt(slope**2+1)
            constant_term = N_x+(slope)*N_y
                
            for id in medial.keys():
                point = face_landmarks.landmark[medial[id]]
                x = int(point.x * width)
                y = int(point.y * height) 
                z = int(point.z * width)
                coord.append(x)
                coord.append(y)
                ai = round(abs(x+(slope)*y-constant_term)/sqrt_slope_square_plus_1,2)
                AIs.append(ai)
                
                #print("{} ({},{},{})".format(id, x, y, z))
                cv2.circle(annotated_image, (x,y), radius=2, thickness=3, color=(255,0,255))
            for id in bilateral.keys():
                point_r = face_landmarks.landmark[bilateral[id][0]]
                point_l = face_landmarks.landmark[bilateral[id][1]]
                x_right = int(point_r.x * width)
                y_right = int(point_r.y * height) 
                z_right = int(point_r.z * width) 
                x_left = int(point_l.x * width)
                y_left = int(point_l.y * height) 
                z_left = int(point_l.z * width) 
                coord.append(x_right)
                coord.append(y_right)
                coord.append(x_left)
                coord.append(y_left)
                #print("{}_right ({},{},{})".format(id, x_right, y_right, z_right))
                #print("{}_left ({},{},{})".format(id, x_left, y_left, z_left))
                ai_1 = abs(x_right+(slope)*y_right-constant_term) - abs(x_left+(slope)*y_left-constant_term)
                ai_2 = abs(slope*(x_left-x_right)-(y_left-y_right))/sqrt_slope_square_plus_1
                ai = round(math.sqrt(pow(ai_1/2,2)+pow(ai_2,2)),2)
                AIs.append(ai)
                cv2.circle(annotated_image, (x_right,y_right), radius=2, thickness=3, color=(255,0,0))
                cv2.circle(annotated_image, (x_left,y_left), radius=2, thickness=3, color=(255,0,0))
                
            df_temp = pd.DataFrame([coord], columns=columns, index=[file])
            df = pd.concat([df, df_temp])
            
            oai = 0.0
            for i in range(len(AIs)):
                oai += AIs[i]*weights[i]
            AIs.append(round(oai/100,2))
            df_temp = pd.DataFrame([AIs], columns=columns_AIs, index=[file])
            df_ai = pd.concat([df_ai, df_temp])

        if not os.path.exists('tmp'): os.mkdir('tmp')
        cv2.imwrite('tmp/annotated_' + file.split('.')[0] + '.png', annotated_image)

# Export to an Excel file with multiple DataFrames
# Create a Pandas Excel writer using XlsxWriter as the engine.
# Write each dataframe to a different worksheet.
# Close the Pandas Excel writer and output the Excel file.
writer = pd.ExcelWriter("oAI_computation.xlsx", engine="xlsxwriter")
df.to_excel(writer, sheet_name="Features")
df_ai.to_excel(writer, sheet_name="AIs+oAI")
writer.save()

#***** Display the final annotated ouput, just for demonstration!! *****
winName = "Face Mesh"
cv2.namedWindow(winName, cv2.WINDOW_AUTOSIZE)
cv2.imshow(winName, annotated_image)

while cv2.waitKey(0) & 0xFF not in [27, ord('q'), ord('Q')]:
    pass
    
cv2.destroyAllWindows()
#***** Display annotated ouput *****